# Time Resolution Scan Analysis

This notebook use the template created with [template_scan_analysis.ipynb](template_scan_analysis.ipynb). follow initial instructions there and run the notebook till the end.

Let us quickly check if the file is really here:

In [ ]:
!ls pulse_template.npz
!ls /mnt/baobab/sst1m/raw/2018/05/25/SST1M_01/SST1M_01_20180525_*.fits.fz

Load the needed libraries.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import os
from digicampipe.io.event_stream import calibration_event_stream
import numpy as np
from glob import glob
from tqdm import tqdm #  progress bar
from ctapipe.instrument import CameraGeometry
from ctapipe.visualization import CameraDisplay
from astropy import units as u
from scipy.stats.distributions import chi2

We read the pulse template data and compute the boolean 2D array which indicates which indexes from the template to match camera data for all possibles offset between 0 an 4 ns (1 line per offset, 1 column per sample).
We plot the pulse for the different offsets.

As large offsets would cut the template pulse, we pad the template with 0 so it does not happen.

In [ ]:
n_sample = 50
max_shift = 4 
template_pulse = np.load('pulse_template.npz')
x = template_pulse['x']
delta_x = x[1] - x[0]
y = template_pulse['y']
dy = template_pulse['dy']
print('loaded', template_pulse.keys(), 'from pulse template.')
print('template with', len(x), 'points from t={:.4f} ns'.format(x[0]),
      'to t={:.4f} ns'.format(x[-1]), 'delta x={:.4f} ns'.format(delta_x))
oversampling = int(np.round(4 / delta_x)) #  samples in data are separated by 4ns
n_offset = int(max_shift * oversampling)
offsets = np.arange(n_offset) * delta_x
print('template was oversampled ', oversampling, 'times compared to data.')
y_max_idx = np.argmax(y)
y_max = y[y_max_idx]

"""
rising_part = np.logical_and(y > 0.1 * y_max, y < 0.9 * y_max)
rising_part = np.logical_and(rising_part, np.arange(len(x)) < y_max_idx)
plt.figure()
plt.errorbar(x[rising_part], y[rising_part], dy[rising_part], 0, '.', ms=0, color='b')
plt.errorbar(x[~rising_part], y[~rising_part], dy[~rising_part], 0, '.', ms=0, color='r')
plt.xlim(np.min(x[rising_part])-1, np.max(x[rising_part])+1)
x=x[rising_part]
y=y[rising_part]
dy=dy[rising_part]
"""

plt.figure()
for idx in range(n_offset):
    offsets_idx = np.arange(idx, len(y), oversampling)
    plt.plot(y[offsets_idx], '-')
    plt.xlabel('sample')
    plt.ylabel('normalized amplitude')
    plt.title('samples for ' + str(n_offset) + ' offsets ({:.1f} ns)'.format(n_offset * delta_x))

plt.figure()
x_padded = np.hstack([np.zeros(max_shift*oversampling), x])
y_padded = np.hstack([np.zeros(max_shift*oversampling), y])
dy_padded = np.hstack([np.ones(max_shift*oversampling)*np.inf, dy])
for idx in range(n_offset):
    offsets_idx = np.arange(idx, len(y_padded), oversampling)
    plt.plot(y_padded[offsets_idx], '-')
    plt.xlabel('sample')
    plt.ylabel('normalized amplitude')
    plt.title('samples for ' + str(n_offset) + ' offsets ({:.1f} ns)'.format(n_offset * delta_x))

Delete old pre-analysed data. To be used if one changes parameters in the next cell and want to have new $\chi^2$ computed.

In [ ]:
output_files = sorted(glob('/home/yves/ctasoft/digicampipe/data/SST1M_01_20180525_*_chi2.npz'))
#output_files = sorted(glob('/mnt/baobab/sst1m/raw/2018/05/25/SST1M_01/SST1M_01_20180525_*_chi2.npz'))
for output_file in output_files:
    os.remove(output_file)

We read the raw camera files.
For each event we normalize the pulse and roughly align it with the template.

We then compute the $\chi^2$ between the measured pulse and the template pulse. We average over all events and store an individual $\chi^2$ value for each AC LEDs'DAC level, each time offset and each pixel.

We store also the mean and the standart deviaton of the integrated pulse over all events for each LED level and each pixel.


In [ ]:
#AC_levels = np.array([305])
#AC_levels = np.array([100, 150, 200, 260, 305, 410, 510, 610, 650, 700, 745])
AC_levels = np.arange(0, 750, 5)
n_level = len(AC_levels)
#input_files = ['/home/yves/ctasoft/digicampipe/data/SST1M_01_20180525_066.fits.fz']
#input_files = sorted(glob('/home/yves/ctasoft/digicampipe/data/SST1M_01_20180525_*.fits.fz'))
input_files = sorted(glob('/mnt/baobab/sst1m/raw/2018/05/25/SST1M_01/SST1M_01_20180525_*.fits.fz'))
print(len(input_files), 'files for', n_level, 'AC levels')
assert len(input_files) == n_level, "ERROR: the number of AC LED levels and the number of files do not match."

n_pixel = 1296
electronic_noise = 1.1 * np.ones(n_pixel) # electronic noise in LSB
zero_pe_integral = 10
gain_integral = 22
chi_squared = np.zeros([n_pixel, n_offset])
pe = np.zeros([n_pixel])
d_pe = np.zeros([n_pixel])
for level_idx, level in enumerate(AC_levels):
    archive_name = input_files[level_idx].replace('.fits.fz', '_chi2.npz').replace(
        '/mnt/baobab/sst1m/raw/2018/05/25/SST1M_01/', '/home/yves/ctasoft/digicampipe/data/'
    )
    if os.path.isfile(archive_name):
        print(input_files[level_idx], 'already_analysed, skiping it.')
        continue
    print('analyzing file', input_files[level_idx], 'for level', level, 'LSB')
    events = calibration_event_stream([input_files[level_idx]])# , max_events=100)
    n_events = np.zeros([n_pixel, n_offset], dtype=np.int)
    integral_events = []
    for e in events:
        adcs = e.data.adc_samples[:, 10:30] - e.data.digicam_baseline[:, None]
        integrals = adcs.sum(axis=1)
        integral_events.append(integrals)
        for idx in range(n_offset):
            offsets_idx = np.arange(idx, len(y_padded), oversampling)
            template_integral = np.sum(y_padded[offsets_idx])
            adc_integral = np.sum(adcs[:, :len(offsets_idx)], axis=1, keepdims=True)
            null_charge = (adc_integral <= 0).flatten()
            adc_integral = adc_integral[~null_charge]
            non_null_noise = electronic_noise[~null_charge]      
            template = y_padded[None, offsets_idx] / template_integral * adc_integral
            template_error = dy_padded[None, offsets_idx] / template_integral * adc_integral
            n_sample_fitted = np.sum(~np.isinf(dy_padded[offsets_idx]))
            squared_template_error = template_error**2 + non_null_noise[:, None]**2
            squared_diff_data_template = (adcs[~null_charge, :len(offsets_idx)] - template)**2
            chi_squared_pixels = np.sum(squared_diff_data_template/squared_template_error, axis=-1)
            chi_squared[~null_charge, idx] += chi_squared_pixels / n_sample_fitted
            n_events[:, idx] += ~null_charge
            assert np.any(np.isnan(squared_template_error)) == False
            assert np.any(np.isnan(squared_diff_data_template)) == False
            assert np.all(squared_template_error > 0)
    chi_squared[n_events>0] /= n_events[n_events>0]
    chi_squared[n_events==0] = np.nan
    integral_events = np.array(integral_events)
    pe = (np.mean(integral_events, axis=0) - zero_pe_integral) / gain_integral
    d_pe = (np.std(integral_events, axis=0) - zero_pe_integral) / gain_integral
    np.savez(archive_name, 
             #integral_events=integral_events, 
             pe=pe, 
             d_pe=d_pe, 
             chi_squared=chi_squared,
             AC_level=level,
             input_file=input_files[level_idx],
             electronic_noise=electronic_noise)
print('done')
pass

We plot the measured charge fluctuations function of the measured charge. The charge is calculated for each pixel and each light level from the integral pulse divided by the gain.
We see the fluctuation is proportional to $\sqrt{p.e.}$ as expected from a Poissonian process until ~700 p.e.

We also plot for each pixel the relation between the LEDs AC level and the measured charge. The limit before saturation (700 p.e.) is shown in red.

In [ ]:
output_files = sorted(glob('/home/yves/ctasoft/digicampipe/data/SST1M_01_20180525_*_chi2.npz'))
#output_files = sorted(glob('/mnt/baobab/sst1m/raw/2018/05/25/SST1M_01/SST1M_01_20180525_*_chi2.npz'))
n_pixel = 1296
n_level = len(output_files)
AC_levels = np.zeros(n_level)
all_integral_events = []
chi_squared = np.zeros([n_level, n_pixel, n_offset])
pe = np.zeros([n_level, n_pixel])
d_pe = np.zeros([n_level, n_pixel])
electronic_noise = np.zeros([n_level, n_pixel])
for i, output_file in enumerate(output_files):
    data = np.load(output_file)
    #integral_events = data['integral_events'].copy()
    pe[i, :] = data['pe'].copy()
    d_pe[i, :] = data['d_pe'].copy()
    chi_squared[i, :, :] = data['chi_squared'].copy()
    AC_levels[i] = data['AC_level'].copy()
    electronic_noise[i, :] = data['electronic_noise'].copy()
del data
print(len(output_files), 'result files loaded.')
#print(chi_squared[1, 16, :])
"""
events_pe = (all_integral_events - zero_pe_integral) / gain_integral
plt.figure(figsize=(12, 6))
plt.hist(events_pe.flatten(), np.arange(-10.5/22, 10, 1/22))
plt.xlabel('charge [p.e.]')
plt.ylabel('counts')
plt.yscale('log', nonposy='clip')
"""

plt.figure(figsize=(12, 6))
#plt.plot(pe.flatten(), d_pe.flatten() ,'+', label='measured')
plt.scatter(pe.flatten(), d_pe.flatten(), 1, np.log10(np.nanmin(chi_squared, axis=2)).flatten(), label='measured')
plt.plot(np.logspace(-1, 3, 100), np.sqrt(np.logspace(-1, 3, 100)), 'k--', label='$\sqrt{p.e.}$')
plt.plot([700, 700], [0, np.max(d_pe)], 'r--')
plt.title('mean charge function of charge variation for all pixels and all levels')
plt.xlabel('mean charge measured [p.e.]')
plt.ylabel('std charge measured [p.e.]')
plt.xscale('log', nonposx='clip')
plt.yscale('log', nonposy='clip')
cbar = plt.colorbar()
#plt.clim(-1, 2)
cbar.set_label('$\log_{10}(\chi^2) for best offset$')
plt.legend()

plt.figure(figsize=(12, 6))
plt.plot(AC_levels, pe, lw=1)
plt.plot([AC_levels[0], AC_levels[-1]], [700, 700], 'r--')
plt.title('charge measured function of DAC level of the LEDs for each pixel')
plt.xlabel('DAC level of AC LEDs')
plt.ylabel('charge measured [p.e.]')
#plt.xlim(0, 400)
plt.ylim(0, 50)

DAC_offset = np.zeros(n_pixel)
for pix in range(n_pixel):
    levels_ok = pe[:, pix] < 50
    poly = np.polyfit(AC_levels[levels_ok], pe[levels_ok, pix], 4)
    poly_shifted = poly.copy()
    poly_shifted[-1] -= 10
    DAC_offset[pix] = np.real(np.roots(poly_shifted)[0])

DAC_offset_ok = np.logical_and(DAC_offset>100, DAC_offset<700)
DAC_offset_ref = np.min(DAC_offset[DAC_offset_ok])
print('DAC_offset_ref:', DAC_offset_ref)
DAC_offset -= DAC_offset_ref
DAC_offset[~DAC_offset_ok] = 0
print(DAC_offset.astype(int).tolist())

geom = CameraGeometry.from_name("DigiCam")
geom.rotate(90 * u.deg)
plt.figure(figsize=(10, 8))
disp = CameraDisplay(geom)
disp.image = DAC_offset
disp.highlight_pixels(~DAC_offset_ok, color='red', linewidth=3)
#disp.set_limits_minmax(0, 700)
disp.add_colorbar()

pass

As we would like the uncertaincy for each pixel function of the charge, we will now fit the offset for all LEDs's AC levels.
Then we plot an histogram of the residuals of the parabola fit.

In [ ]:
chi_squared_max = 100
points_rel = np.arange(-15, 16, 1)

n_point_fit = len(points_rel)
polys = np.ones([3, n_level, n_pixel]) * np.nan
residuals = np.ones([n_level, n_pixel]) * np.nan
index_offset_rel = np.ones([n_level, n_pixel]) * np.nan
plt.figure(figsize=(12, 6))
for level_idx in range(n_level):
    bad_pixels = np.any(np.isnan(chi_squared[level_idx, :, :]), axis=-1)
    saturated_pixels = pe[level_idx, :] > 800
    dark_pixels = pe[level_idx, :] < 0.5
    pixels_ok_bool = np.logical_and(~bad_pixels, ~saturated_pixels)
    pixels_ok_bool = np.logical_and(pixels_ok_bool, ~dark_pixels)
    #pixels_ok_bool = np.logical_and(pixels_ok_bool, ~problematic_pixels)
    pixels_ok = np.arange(n_pixel)[pixels_ok_bool]
    # print('level', AC_levels[level_idx], ':', len(pixels_ok), '/', n_pixel, 'fits are ok')
    chi_squared_level = chi_squared[level_idx, pixels_ok, :]
    index_best_offsets = np.nanargmin(chi_squared_level, axis=-1)
    index_best_offsets[index_best_offsets < -np.min(points_rel)] = -np.min(points_rel)
    index_best_offsets[index_best_offsets >= n_offset - np.max(points_rel)] =  n_offset - np.max(points_rel) -1
    fit_points_bool = np.zeros([len(pixels_ok), n_offset], dtype=bool)
    for k in points_rel:
        fit_points_bool = np.logical_or(fit_points_bool, np.eye(n_offset,k=k)[index_best_offsets])
    # we fit parabolas
    chi_squared_fit = chi_squared_level[fit_points_bool].reshape(len(pixels_ok), n_point_fit)
    if chi_squared_fit.shape[0] == 0:
        print("no good pixel found for level", AC_levels[level_idx])
        continue
    polys_level, residuals_level, _, _, _ = np.polyfit(points_rel*delta_x, chi_squared_fit.transpose(), 2, full=True)
    polys[:, level_idx, pixels_ok] = polys_level
    residuals[level_idx, pixels_ok] = residuals_level
    index_offset_rel[level_idx, pixels_ok] = index_best_offsets
    #plot the fitted parabollas
    #plt.plot(points_rel*delta_x, chi_squared_fit.transpose(), '-', lw=1)
    #plt.xlabel('offset [ns]')
    #plt.ylabel('$\chi^2 /(N-1)$')
    #plt.ylim([0, chi_squared_max])
polys = polys.reshape(3, -1)
offset_fit_rel = np.ones([n_level* n_pixel]) * np.nan
offset_fit_rel = (-0.5 * polys[1, :] / polys[0, :])
offset_fit = offset_fit_rel.reshape(n_level, n_pixel) + index_offset_rel * delta_x
d_offset_fit = np.ones([n_level* n_pixel]) * np.nan
d_offset_fit = (np.sqrt(1./polys[0, :]))
d_offset_fit = d_offset_fit.reshape(n_level, n_pixel)
chi_squared_fit_min = polys[2, :].copy()
residuals = residuals.reshape(-1)

"""
# we plot the residuals
plt.figure(figsize=(12, 6))
h, xh, _ = plt.hist(residuals[~np.isnan(residuals)], np.logspace(-1, 3, 200))
plt.xlabel('residuals')
plt.ylabel('counts')
plt.xscale('log', nonposx='clip')
plt.yscale('log', nonposy='clip')
"""
fit_ok = ~np.isnan(residuals)
# residuals function of min chi2
fig = plt.figure(figsize=(12, 12))
axes = fig.subplots(2, sharex=True)
h0=axes[0].scatter(pe.flatten()[fit_ok], offset_fit.flatten()[fit_ok], 1 ,d_offset_fit.flatten()[fit_ok])#, vmin=7, vmax=12)
cbar0 = plt.colorbar(h0, ax=axes[0])
cbar0.set_label('offset uncertaincy [ns]')
axes[0].set_xscale('log', nonposx='clip')
axes[0].set_xlabel('charge [p.e.]')
axes[0].set_ylabel('fitted offset [ns]')
axes[0].set_title('fitted offset function of charge for all pixels and light levels')
#axes[0].set_ylim([7, 15])
h1 = axes[1].scatter(pe.flatten()[fit_ok], offset_fit.flatten()[fit_ok], 1, chi_squared_fit_min.flatten()[fit_ok], vmin=0, vmax=10)
cbar1 = plt.colorbar(h1, ax=axes[1])
cbar1.set_label('$\chi^2_{min}$')
axes[1].set_xscale('log', nonposx='clip')
axes[1].set_xlabel('charge [p.e.]')
axes[1].set_ylabel('fitted offset [ns]')
axes[1].set_title('fitted offset function of charge for all pixels and light levels')
#axes[1].set_ylim([7, 15])
pass

We calculate the offset and the uncertaincy as previously but for all light levels giving a reasonable fit.
Some aditional attention must be taken as we are fitting on less than optimal data.

There are for each pixel ~3 functional fits, and they are consistent with each other.

In [ ]:
#We discard the fits where the residuals are too large.
fit_ok = chi_squared_fit_min < 10
fit_ok = np.logical_and(fit_ok, chi_squared_fit_min > .1)
fit_ok = np.logical_and(fit_ok, chi_squared_fit_min < 10)
fit_ok = np.logical_and(fit_ok, offset_fit.flatten() > 7)
fit_ok = np.logical_and(fit_ok, offset_fit.flatten() < 12)
print(np.sum(fit_ok), '/', len(fit_ok.flatten()), 'fits succeded')
delta_chi_square = np.polyval(polys, offset_fit_rel - d_offset_fit.flatten()) - chi_squared_fit_min

# Now we check the hypothesis made to calculate the uncertaincy still holds. When too much off, we discard the fit.
fit_ok = np.logical_and(fit_ok, delta_chi_square > 0.8)
offset_fit_rel[~fit_ok] = np.nan
offset_fit.flatten()[~fit_ok] = np.nan
d_offset_fit.flatten()[~fit_ok] = np.nan
chi_squared_fit_min[~fit_ok] = np.nan
print(np.sum(fit_ok), '/', len(fit_ok), 'fits succeded after further checks')
std_offset_fitted = np.nanstd(offset_fit, axis=0)
std_offset_fitted_mean = np.nanmean(std_offset_fitted[std_offset_fitted>0])
std_offset_fitted_min = np.nanmin(std_offset_fitted[std_offset_fitted>0])
print('the average over all pixels of the deviation of the fitted offset over the AC levels is:', std_offset_fitted)

plt.figure(figsize=(12, 6))
plt.plot(np.arange(len(fit_ok))[fit_ok], delta_chi_square[fit_ok], '+')
plt.plot(np.arange(len(fit_ok))[~fit_ok], delta_chi_square[~fit_ok], '+')
plt.plot([0, len(fit_ok)], [1, 1], 'k--')
plt.title('$\Delta \chi^2$ at $1 \sigma$ (should be ~1)')
plt.xlabel('fit #')
plt.ylabel('$\Delta \chi^2$')
plt.ylim(0, 1.1)

plt.figure(figsize=(12, 6))
plt.plot(offset_fit.transpose(), '+')
plt.xlabel('pixel #')
plt.ylabel('offset [ns]')

plt.figure(figsize=(12, 6))
plt.plot(d_offset_fit.transpose(), '+')
plt.plot([0, d_offset_fit.shape[1]], std_offset_fitted_mean * np.ones(2), '--k')
plt.plot([0, d_offset_fit.shape[1]], std_offset_fitted_min * np.ones(2), '--k')
plt.xlabel('pixel #')
plt.ylabel('error on offset [ns]')
plt.yscale('log', nonposy='clip')

pass

Finally we plot :
- the uncertaincy function of the LEDs AC level
- the uncertaincy function of the charge

It is very uniform around 0.22 ns in the 20 - 700 p.e. range.

In [ ]:
plt.figure(figsize=(12, 6))
pixels_plot = (np.ones([len(AC_levels), 1]) * np.arange(n_pixel))
levels_plot = (np.ones([n_pixel, 1]) * AC_levels).transpose()
plt.scatter(pixels_plot.flatten(), levels_plot.flatten(), 5, np.log10(pe).flatten(), '+')
plt.xlabel('pixel #')
plt.ylabel('LED AC level')
cbar = plt.colorbar()
cbar.set_label('$\log_{10}$(charge [p.e.])')

plt.figure(figsize=(12, 6))
plt.scatter(np.ones([n_pixel, 1]) * AC_levels, d_offset_fit, 5, np.log10(pe), '+')
plt.xlabel('LED AC level')
plt.ylabel('error on offset [ns]')
cbar = plt.colorbar()
cbar.set_label('$\log_{10}$(charge [p.e.])')

plt.figure(figsize=(12, 6))
plt.scatter(pe.flatten()[fit_ok], d_offset_fit.flatten()[fit_ok],5, np.log10(chi_squared_fit_min[fit_ok]), '+')
plt.xlim([1, 1000])
plt.ylim([0.1, 10])
cbar = plt.colorbar()
cbar.set_label('$\log_{10}(\chi^2)$')
plt.xscale('log', nonposx='clip')
plt.yscale('log', nonposy='clip')
plt.xlabel('measured charge [p.e.]')
plt.ylabel('error on offset [ns]')
plt.title('timing resolution function of charge')
pass

Lets try to understand the patern of the measured offset.

No obvious correlation with sectors nor boards nor pixel index in boards.

In [ ]:
best_offsets = np.nan*np.ones(n_pixel)
best_d_offsets = np.nan*np.ones(n_pixel)
bad_pixels = np.all(np.isnan(d_offset_fit), axis=0)
best_levels = np.nanargmin(d_offset_fit[:, ~bad_pixels], axis=0)
best_offsets[~bad_pixels]  = offset_fit[best_levels, ~bad_pixels]
best_d_offsets[~bad_pixels] = d_offset_fit[best_levels, ~bad_pixels]
geom = CameraGeometry.from_name("DigiCam")
geom.rotate(90 * u.deg)

plt.scatter(np.arange(n_pixel), best_offsets, 5,  best_d_offsets)
plt.figure(figsize=(10, 8))
disp = CameraDisplay(geom)
disp.image = best_offsets
disp.highlight_pixels(bad_pixels, color='red', linewidth=3)
#disp.set_limits_minmax(np.min(offset_fit), np.max(offset_fit))
disp.set_limits_minmax(7, 12)
disp.add_colorbar()
plt.title('measured time offset [ns]')
#disp.overlay_pixels_id()

plt.figure(figsize=(10, 8))
disp = CameraDisplay(geom)
disp.image = best_d_offsets
disp.set_limits_minmax(0, 0.3)
disp.highlight_pixels(bad_pixels, color='red', linewidth=3)
cbar = disp.add_colorbar()
plt.title('uncertaincy on time offset [ns]')
pass

In [ ]:
patches_to_pixels[0,:]

In [ ]:
pixels_to_patches[755]